<a href="https://colab.research.google.com/github/MohamedAziz15/Transformer-Archticture-from-Scratch/blob/main/Building_Transformer_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
text = "time flies like an arrow"
show(model, "bert", tokenizer, text, display_mode="light",
layer=0, head=8)

In [ ]:
inputs = tokenizer(text, return_tensors="pt",
add_special_tokens=False)
inputs.input_ids

In [ ]:
from torch import nn
from transformers import AutoConfig
config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

In [ ]:
import torch
from math import sqrt
query = key = value = inputs_embeds
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
scores.size()

In [ ]:
import torch.nn.functional as F
weights = F.softmax(scores, dim=-1)
weights.sum(dim=-1)

In [ ]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

In [ ]:
def scaled_dot_product_attention(query, key, value):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
  weights = F.softmax(scores, dim=-1)
  return torch.bmm(weights, value)

In [ ]:
class AttentionHead(nn.Module):
  def __init__(self, embed_dim, head_dim):
      super().__init__()
      self.q = nn.Linear(embed_dim, head_dim)
      self.k = nn.Linear(embed_dim, head_dim)
      self.v = nn.Linear(embed_dim, head_dim)
  def forward(self, hidden_state):
      attn_outputs = scaled_dot_product_attention(
      self.q(hidden_state), self.k(hidden_state),
      self.v(hidden_state))
      return attn_outputs

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, config):
      super().__init__()
      embed_dim = config.hidden_size
      num_heads = config.num_attention_heads
      head_dim = embed_dim // num_heads
      self.heads = nn.ModuleList([AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]  )
      self.output_linear = nn.Linear(embed_dim, embed_dim)
  def forward(self, hidden_state):
      x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
      x = self.output_linear(x)
      return x


In [ ]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embeds)
attn_output.size()

In [ ]:
from bertviz import head_view
from transformers import AutoModel
model = AutoModel.from_pretrained(model_ckpt,
output_attentions=True)
sentence_a = "time flies like an arrow"
sentence_b = "fruit flies like a banana"
viz_inputs = tokenizer(sentence_a, sentence_b,
return_tensors='pt')
attention = model(**viz_inputs).attentions
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])
head_view(attention, tokens, sentence_b_start, heads=[8])


In [ ]:
class FeedForward(nn.Module):
  def __init__(self, config):
      super().__init__()
      self.linear_1 = nn.Linear(config.hidden_size,
      config.intermediate_size)
      self.linear_2 = nn.Linear(config.intermediate_size,
      config.hidden_size)
      self.gelu = nn.GELU()
      self.dropout = nn.Dropout(config.hidden_dropout_prob)
  def forward(self, x):
      x = self.linear_1(x)
      x = self.gelu(x)
      x = self.linear_2(x)
      x = self.dropout(x)
      return x

In [ ]:
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_outputs)
ff_outputs.size()